# Data Preprocessing: Save data for analysis

Previous steps shown that raw data that were collected from the Internet, need some initial pre-processing before analysis. It includes limiting data to only records that will be analyzed, removing outliers and adding some frequently used columns.

What is more, datasets with aggregated daily and hourly values will be calculated and saved to avoid doing the same operation multiple times in the future.

In [ ]:
import os
os.chdir('..')

In [1]:
import pandas as pd
import numpy as np
import functions as f

In [6]:
df_rentals = pd.read_parquet('data/db_rentals.parquet')

In [2]:
df_distance = pd.read_parquet('data/db_stations_distance.parquet')

C:\Users\Klaudia\Anaconda3\envs\bikerental\lib\site-packages\pyarrow\pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\Klaudia\Anaconda3\envs\bikerental\lib\site-packages\pyarrow\pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\Klaudia\Anaconda3\envs\bikerental\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
df_weather = pd.read_parquet('data/db_weather.parquet')

## Subset of data for analysis

The subset will include data limited to years 2015-2018, only from Washington D.C. The dataset will be enhanced with information about the distance and speed of ride.

In [7]:
df_subset = df_rentals.copy()
df_subset['year'] = df_subset['start_date'].dt.year

In [8]:
df_subset = df_subset[(df_subset['year']>=2015) & (df_subset['year']<=2018) & (df_subset['city']=='D.C.') & (df_subset['member_type'].isin(['Member', 'Casual']))].reset_index(drop=True)

In [8]:
df_subset['year_month'] = df_subset['start_date'].dt.to_period('M')
df_subset['year_month'] = pd.PeriodIndex(df_subset['year_month']).to_timestamp()
df_subset['month'] = df_subset['start_date'].dt.month
df_subset['weekday'] = df_subset['start_date'].dt.weekday

In [9]:
df_subset = df_subset.merge(df_distance, how='left', left_on=['start_station_id', 'end_station_id'], right_on=['station_a', 'station_b'])

In [10]:
drop_cols = [c for c in df_subset.columns if '_a' in c or '_b' in c]
df_subset = df_subset.drop(drop_cols, axis=1)
df_subset['speed'] = df_subset['distance'] / (df_subset['duration']/3600)

In [14]:
df_subset.sample(3)

,rental_id,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,duration,city,year,year_month,month,weekday,distance,speed
4843289,14338403,2016-09-09 08:20:18,2016-09-09 08:39:31,31802,31641,W21599,Member,1153,D.C.,2016,2016-09-01,9,4,3.332121,10.403846
78581,8927427,2015-01-22 17:38:19,2015-01-22 18:00:28,31511,31245,W20689,Member,1328,D.C.,2015,2015-01-01,1,3,3.232119,8.761768
1459222,10487884,2015-07-20 15:27:23,2015-07-20 15:40:20,31236,31236,W21373,Member,776,D.C.,2015,2015-07-01,7,0,NaN,NaN


In [15]:
#Remove speed outliers according to the analysis
# 1st step - divide by long and short rides

upper_z = df_subset['duration'].mean() + 3 * df_subset['duration'].std()
df_subset['ride_length'] = 'normal'
df_subset.loc[df_subset['duration']>upper_z, 'ride_length'] = 'long'

df_subset.loc[df_subset['ride_length']=='long', ['distance', 'speed']] = np.nan

In [16]:
# 2nd step - remove speedy rides
# put this function to functions file

def remove_station_rides(df, station_id, min_date='1990-01-01'):
    df.loc[(df['start_station_id']==station_id) | (df['end_station_id']==station_id) & (df['start_date']<min_date), ['distance', 'speed']] = np.nan
    return df

df_subset = remove_station_rides(df_subset, 31064)
df_subset = remove_station_rides(df_subset, 31062)
df_subset = remove_station_rides(df_subset, 31262, '2013-01-01')
df_subset = remove_station_rides(df_subset, 32006, '2013-09-01')
df_subset = remove_station_rides(df_subset, 31062, '2013-09-01')
df_subset = remove_station_rides(df_subset, 31052, '2012-01-01')
df_subset = remove_station_rides(df_subset, 31266, '2013-01-01')

In [17]:
speedy_dates = f.round_byTime(df_subset[df_subset['speed']>25], 'D').sort_values(by='rentals_count', ascending=False)
            
dates = np.array(speedy_dates[speedy_dates['rentals_count']>10]['start_date'].values, dtype='datetime64[D]')
df_subset['date'] = df_subset['start_date'].dt.date
df_subset.loc[df_subset['date'].isin(dates), ['distance', 'speed']] = np.nan

df_subset.loc[df_subset['speed']>40, ['distance', 'speed']] = np.nan

In [18]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12123932 entries, 0 to 12123931
Data columns (total 17 columns):
rental_id           int64
start_date          datetime64[ns]
end_date            datetime64[ns]
start_station_id    int64
end_station_id      int64
bike_id             object
member_type         object
duration            int64
city                object
year                int64
year_month          datetime64[ns]
month               int64
weekday             int64
distance            float64
speed               float64
ride_length         object
date                object
dtypes: datetime64[ns](3), float64(2), int64(7), object(5)
memory usage: 1.6+ GB


In [19]:
df_subset.to_parquet('data/data_bikerental.parquet', index=False)

## Weather data cleanup

In [17]:
df_weather = df_weather[df_weather['temperature']<45]
df_weather = df_weather[df_weather['wind']<400]

In [18]:
df_weather.to_parquet('data/data_weather.parquet')

C:\Users\Klaudia\Anaconda3\envs\bikerental\lib\site-packages\pyarrow\pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


## Hourly data

Group bike rentals data by hour and combine it with information about the weather.

In [19]:
df_weather['datetime_round'] = df_weather['datetime'].dt.round(freq='H')
df_weather.sample(3)

,datetime,date,time,temperature,relative_temperature,wind,relative_humidity,pressure,clouds,rain,snow,thunder,datetime_round
51991,2016-02-12 19:52:00,2016-02-12,19:52,-2,-5,150,0.51,1016,cloudy,0,0,0,2016-02-12 20:00:00
34291,2014-06-30 14:52:00,2014-06-30,14:52,31,32,170,0.49,1016,partly cloudy,0,0,0,2014-06-30 15:00:00
46219,2015-08-05 16:52:00,2015-08-05,16:52,34,33,290,0.30,1013,partly cloudy,0,0,0,2015-08-05 17:00:00


In [20]:
df_rentals = df_rentals[(df_rentals['city']=='D.C.') & (df_rentals['member_type']!='Unknown')]

In [21]:
df_hourly = f.round_byTime(df_rentals, 'H') \
            .merge(df_weather, how='left', left_on='start_date', right_on='datetime_round') \
            .drop(['datetime', 'time', 'datetime_round'], axis=1)
        
df_hourly_byMember = f.round_byTime(df_rentals, 'H', other_cols=['member_type']) \
            .merge(df_weather, how='left', left_on='start_date', right_on='datetime_round') \
            .drop(['datetime', 'time', 'datetime_round'], axis=1)

In [22]:
df_hourly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88190 entries, 0 to 88189
Data columns (total 13 columns):
start_date              88190 non-null datetime64[ns]
rentals_count           88190 non-null int64
avg_duration            87285 non-null float64
date                    85023 non-null datetime64[ns]
temperature             85023 non-null float64
relative_temperature    85023 non-null float64
wind                    85023 non-null float64
relative_humidity       85023 non-null float64
pressure                85023 non-null float64
clouds                  85023 non-null object
rain                    85023 non-null float64
snow                    85023 non-null float64
thunder                 85023 non-null float64
dtypes: datetime64[ns](2), float64(9), int64(1), object(1)
memory usage: 9.4+ MB


In [23]:
df_hourly.to_parquet('data/data_hourly.parquet', index=False)
df_hourly_byMember.to_parquet('data/data_hourly_byMember.parquet', index=False)

## Data daily

In [24]:
df_daily = f.round_byTime(df_rentals, 'D')
df_daily_byMember = f.round_byTime(df_rentals, 'D', other_cols=['member_type'])

In [25]:
df_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3084 entries, 0 to 3083
Data columns (total 3 columns):
start_date       3084 non-null datetime64[ns]
rentals_count    3084 non-null int64
avg_duration     3080 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 72.4 KB


In [26]:
df_daily.to_parquet('data/data_daily.parquet', index=False)
df_daily_byMember.to_parquet('data/data_daily_byMember.parquet', index=False)